Recherche et extraction des entités nommées avec Transformers.
Modèle de language utilisé: (Camem)BERT.

In [2]:
# @title Installation des bibliothèques, connexion au Google Drive

"""Marina Giardinetti
marina.giardinetti@gmail.com
Google Colab
2024"""

from google.colab import drive
drive.mount('/content/drive')
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
import re
import os
import glob
import pandas as pd

tokenizer = AutoTokenizer.from_pretrained("Jean-Baptiste/camembert-ner")
model = AutoModelForTokenClassification.from_pretrained("Jean-Baptiste/camembert-ner")

##### Process text sample (from wikipedia)

nlp = pipeline('ner', model=model, tokenizer=tokenizer, aggregation_strategy="simple")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


tokenizer_config.json:   0%|          | 0.00/269 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/892 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/811k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/210 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [3]:
# @title Chargement du corpus
liste_ner =[]

working_dir= '/content/drive/My Drive/seance_22_04/' #@param {type:"string"}
corpus = '/content/drive/My Drive/seance_22_04/test/' #@param {type:"string"}

# Afficher le nombre de textes

textes = os.listdir(corpus)
print('Nombre de textes chargés dans le corpus: ', len(textes))


Nombre de textes chargés dans le corpus:  1


In [5]:
# @title Recherche et extraction des entités nommées
liste_doc=[]
textes = os.listdir(corpus)
for texte in textes:
  content = open(''.join([corpus, texte]))
  content = content.read()
  nom_doc = os.path.basename(texte)

  res = [x for x in re.split("[//.|//!|//?]", content) if x!=""]
  for sentence in res:
    ner = nlp(sentence)
    for dict_ner in ner:
      dict_ner.update({'document':nom_doc})
    liste_ner.append(ner)
    liste_doc.append(nom_doc)
liste_ner2 =[]
for el in liste_ner:
  for el2 in el:
    liste_ner2.append(el2)

df = pd.DataFrame.from_records(liste_ner2)

#@markdown Indiquer le taux de sureté minimal requis pour chaque entité detectée
taux_de_precision = "0.95" #@param {type:"string"}

df_pers = df.loc[(df['entity_group'] == 'PER') & (df['score'] >= float(taux_de_precision))]
df_pers.to_excel(''.join([working_dir, 'bert_pers.xlsx']))


df_loc = df.loc[(df['entity_group'] == 'LOC') & (df['score'] >= float(taux_de_precision))]
df_loc.to_excel(''.join([working_dir, 'bert_lieux.xlsx']))
